# This notebook will generate the synthetic data required as an input to our Facebook/Prophet model
- Some noise will be added to the data to reflect real data!
- This notebook output will be saved in the data folder (RAW)

## Two tables will be generated:
- The first table will attach every product to a cluster of products (5 clusters A, B, C, D and E will be used):

| Product_code  |  Associated cluster |
|---|---|
| CLA01  |  A  |
| CLA02  |   E  |
| CLB01  |   A  |

The product code 3 first characters define the client : CLA : Client A , CLB : Client B etc.

- The second table is the actual Sales history table:

| Product_code  | Date  |  Quantity |
|---|---|---|
|  CLB01 | 25/07/2019  | 1,000  |
|  CLB01 | 19/07/2019  |  1,500 |
|   CLA02 | 23/07/2019  |  10,000 |

### Product table generation

#### 1. Import the famous pandas and numpy libraries

In [2]:
import pandas as pd
import numpy as np

In [3]:
#This sets the seed to always generate the same data

from numpy.random import RandomState
random_state=RandomState(9999)

In [23]:
#Input parameters

number_of_clients=5
min_number_of_products_per_client=1
max_number_of_products_per_client=6
number_of_clusters=5

In [5]:
number_of_products_list=random_state.randint(min_number_of_products_per_client,
                                             max_number_of_products_per_client,
                                             number_of_clients)

In [6]:
number_of_products_list

array([2, 5, 2, 4, 1])

In [7]:
def letter_range(start, stop="{", step=1):
    """Yield a range of lowercase letters.""" 
    for ord_ in range(ord(start.upper()), ord(stop.upper()), step):
        yield chr(ord_)

In [8]:
clients_list=['CL'+ l for l in letter_range(chr(97),chr(97+number_of_clients))]

In [9]:
clients_list

['CLA', 'CLB', 'CLC', 'CLD', 'CLE']

In [18]:
product_codes=[(client,client+str(i)) for number_of_products,client in zip(number_of_products_list,clients_list) 
               for i in range(number_of_products) ]

In [19]:
product_codes

[('CLA', 'CLA0'),
 ('CLA', 'CLA1'),
 ('CLB', 'CLB0'),
 ('CLB', 'CLB1'),
 ('CLB', 'CLB2'),
 ('CLB', 'CLB3'),
 ('CLB', 'CLB4'),
 ('CLC', 'CLC0'),
 ('CLC', 'CLC1'),
 ('CLD', 'CLD0'),
 ('CLD', 'CLD1'),
 ('CLD', 'CLD2'),
 ('CLD', 'CLD3'),
 ('CLE', 'CLE0')]

In [35]:
index = pd.MultiIndex.from_tuples(product_codes, names=['first', 'second'])

In [36]:
product_table=pd.Series(random_state.randint(0,number_of_clusters, len(product_codes)), index=index)

In [46]:
product_table.rename('Cluster',inplace=True)

first  second
CLA    CLA0      1
       CLA1      3
CLB    CLB0      0
       CLB1      1
       CLB2      2
       CLB3      4
       CLB4      2
CLC    CLC0      1
       CLC1      2
CLD    CLD0      0
       CLD1      4
       CLD2      2
       CLD3      3
CLE    CLE0      3
Name: Cluster, dtype: int64

In [47]:
product_table

first  second
CLA    CLA0      1
       CLA1      3
CLB    CLB0      0
       CLB1      1
       CLB2      2
       CLB3      4
       CLB4      2
CLC    CLC0      1
       CLC1      2
CLD    CLD0      0
       CLD1      4
       CLD2      2
       CLD3      3
CLE    CLE0      3
Name: Cluster, dtype: int64

#### Now building the Sales table

In [55]:
products_list=product_table.index.levels[1].values

In [56]:
# This sets the frequencies in days for each product which will be used for the dates generator
products_frequencies=random_state.randint(1,365.25//2,len(products_list))

In [57]:
products_frequencies

array([141,  75,  74, 147, 114,  91,  69, 161,  44, 104, 171, 103,  27,
        89])

In [65]:
start_date='1/1/2018'
end_date='26/07/2019'

In [89]:
products_dates_list=[pd.date_range(start=start_date,
             end=end_date,
              freq=str(freq)+'D') for freq in products_frequencies]

In [107]:
products_dates_list[:2]

[DatetimeIndex(['2018-01-01', '2018-05-22', '2018-10-10', '2019-02-28',
                '2019-07-19'],
               dtype='datetime64[ns]', freq='141D'),
 DatetimeIndex(['2018-01-01', '2018-03-17', '2018-05-31', '2018-08-14',
                '2018-10-28', '2019-01-11', '2019-03-27', '2019-06-10'],
               dtype='datetime64[ns]', freq='75D')]

In [91]:
quantities_list=random_state.randint(1000,10000,len(products_list))

In [93]:
quantities_list

array([8865, 2496, 4588, 5399, 1130, 8850, 5854, 9909, 2320, 9895, 7857,
       9631, 3851, 5966])

In [163]:
exact_sales_df=pd.DataFrame([(prod,date.ctime(),quant) for prod,quant,dates_list in zip(products_list,quantities_list,products_dates_list) 
              for date in dates_list],
            columns=['product_code',
                     'date',
                    'quantity'])

In [164]:
exact_sales_df['date']=pd.to_datetime(exact_sales_df['date'])

In [165]:
exact_sales_df.head()

,product_code,date,quantity
0,CLA0,2018-01-01,8865
1,CLA0,2018-05-22,8865
2,CLA0,2018-10-10,8865
3,CLA0,2019-02-28,8865
4,CLA0,2019-07-19,8865


In [166]:
days_noise,quantities_noise=10,0.1

In [167]:
sales_df=exact_sales_df.copy()

In [168]:
sales_df['random_date_delta']=np.array([pd.to_timedelta(str(i)+'D') 
                                         for i in random_state.randint(-days_noise,
                                                                       days_noise,sales_df.shape[0])])

In [169]:
sales_df.head()

,product_code,date,quantity,random_date_delta
0,CLA0,2018-01-01,8865,6 days
1,CLA0,2018-05-22,8865,-10 days
2,CLA0,2018-10-10,8865,6 days
3,CLA0,2019-02-28,8865,7 days
4,CLA0,2019-07-19,8865,-10 days


In [170]:
sales_df['noisy_date']=sales_df['date']+sales_df['random_date_delta']

In [171]:
sales_df.head()

,product_code,date,quantity,random_date_delta,noisy_date
0,CLA0,2018-01-01,8865,6 days,2018-01-07
1,CLA0,2018-05-22,8865,-10 days,2018-05-12
2,CLA0,2018-10-10,8865,6 days,2018-10-16
3,CLA0,2019-02-28,8865,7 days,2019-03-07
4,CLA0,2019-07-19,8865,-10 days,2019-07-09


In [172]:
sales_df['random_qty_delta']=sales_df['quantity']*random_state.uniform(-quantities_noise,quantities_noise,
                                                                                   sales_df.shape[0])

In [173]:
sales_df.head()

,product_code,date,quantity,random_date_delta,noisy_date,random_qty_delta
0,CLA0,2018-01-01,8865,6 days,2018-01-07,-748.954989
1,CLA0,2018-05-22,8865,-10 days,2018-05-12,-216.395953
2,CLA0,2018-10-10,8865,6 days,2018-10-16,-747.573181
3,CLA0,2019-02-28,8865,7 days,2019-03-07,244.582150
4,CLA0,2019-07-19,8865,-10 days,2019-07-09,-269.123981


In [174]:
sales_df['noisy_quantity']=sales_df['quantity']+sales_df['random_qty_delta']

In [175]:
sales_df.head()

,product_code,date,quantity,random_date_delta,noisy_date,random_qty_delta,noisy_quantity
0,CLA0,2018-01-01,8865,6 days,2018-01-07,-748.954989,8116.045011
1,CLA0,2018-05-22,8865,-10 days,2018-05-12,-216.395953,8648.604047
2,CLA0,2018-10-10,8865,6 days,2018-10-16,-747.573181,8117.426819
3,CLA0,2019-02-28,8865,7 days,2019-03-07,244.582150,9109.582150
4,CLA0,2019-07-19,8865,-10 days,2019-07-09,-269.123981,8595.876019


In [176]:
noisy_sales_df=sales_df[['product_code',
                        'noisy_date',
                        'noisy_quantity']]

In [178]:
noisy_sales_df.head()

,product_code,noisy_date,noisy_quantity
0,CLA0,2018-01-07,8116.045011
1,CLA0,2018-05-12,8648.604047
2,CLA0,2018-10-16,8117.426819
3,CLA0,2019-03-07,9109.582150
4,CLA0,2019-07-09,8595.876019


In [179]:
noisy_sales_df.shape

(109, 3)

In [182]:
noisy_sales_df.to_csv('../data/raw/sales_table.csv',
                     index=False)

In [192]:
products_clusters=pd.DataFrame(product_table.droplevel(0))

In [201]:
products_clusters.rename_axis('product_code',axis=0,inplace=True)

In [202]:
products_clusters

,Cluster
product_code,
CLA0,1
CLA1,3
CLB0,0
CLB1,1
CLB2,2
CLB3,4
CLB4,2
CLC0,1
CLC1,2


In [203]:
sales_cluster_df=noisy_sales_df.join(products_clusters,how='outer',on='product_code')

In [205]:
sales_cluster_df.shape

(109, 4)

In [206]:
sales_cluster_df.head()

,product_code,noisy_date,noisy_quantity,Cluster
0,CLA0,2018-01-07,8116.045011,1
1,CLA0,2018-05-12,8648.604047,1
2,CLA0,2018-10-16,8117.426819,1
3,CLA0,2019-03-07,9109.582150,1
4,CLA0,2019-07-09,8595.876019,1


In [207]:
sales_cluster_df.Cluster.value_counts()

3    37
2    34
0    14
1    13
4    11
Name: Cluster, dtype: int64